Welcome!

If you haven't read the README.md file at my git, start there. Check the pdf "Matching VBF-quarks with Jets", also, to get some background and purpose for this code. You should know what is bGenPart (best GenPart), bGenJet and bJet, etc. Check also histogram pdfs in different directories.

This "main.ipynb" python file is the main file and the rest are runned from here by %run or %load (and run) commands.

The code structure from the beginning to end:

    1) creating the RDataframe from data files,
    2) defining variables, functions and filtering the data to interesting events only,
    3) optional

The third step ("optional") can be anything but specially (there are different functions and python files to run depending on the choice):

    1) print and study the matchings (bGenPart-->bGenJet-->bJet) success rates and results (the "printStats" -files and functions),
    2) study histograms of different variables (there are already many histograms in the directories),
    3) print and study the events one by one and print interesting properties about it (the "printEvents" -files and functions),
    4) improve finding of the VBF jets with detector data towards "strong Jets" using pandas dataframe, and sklearn machine learning algorithms
    
    
I recommend studying the code below well enough before doing "Restart & Run All". You might want to study only one of the four things listed above and comment out the other %run and %load commands.

Notice that the dataset events can be filtered with (at least) two different ways: fatJetFilter (loose filter) and viljaFilter (SalomaaMod recommended).

If help is needed, I can be connected.

BR. Santeri Salomaa

In [1]:
# Random tip:
# Visualizing the data columns helps in many situations. You can do this easily, for example using pandas dataframe
# (df0 = RDataframe, npdf = numpy dataframe, pddf = pandas dataframe)

# Example:
# npdf = df0.AsNumpy(['column_1', 'column_2'])
# pddf = pd.DataFrame(npdf)
# pddf.iloc[:5]

# -------------------------------------

###### All NanoAOD v7 variables are listed here: https://cms-nanoaod-integration.web.cern.ch/integration/master/mc94X_doc.html#LHE
###### List of all custom NanoAODv7 files is at: https://cmsweb.cern.ch/das/request?view=list&limit=150&instance=prod%2Fphys03&input=%2F*%2F*-NanoTuples-30Apr2020_*%2FUSER

# -------------------------------------

## All imports:

In [33]:
# Time can be imported for checking the runtime

import time
t_init = time.time()

In [3]:
import ROOT

Welcome to JupyROOT 6.24/00


In [4]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

# For histograms
from ROOT import TLine

# Imports for machine learning
import sklearn as sk
import matplotlib
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [5]:
# Batch mode (do not open windows while running)
ROOT.gROOT.SetBatch(True)

# Enable multi-threading with 4 threads 
#ROOT.ROOT.EnableImplicitMT(4)

# Choose the dataset and filtering of the events.

In [6]:
# Choose the dataset and filtering of the events. You can delete this and make own (better/simpler) method of choosing the data if you want.
# Look up the next cells to know wat these "flags" actually do.
# Here the data is "chosen" and after this the code uses if-functions to handle and make the dataframe.
# HH = di-Higgs data and WW = di-W-boson data (tt and ll is the polarisation of the bosons)
#********************************
# Datafile (choose one and specify if needed, f.ex. WW = True and WW_tt = True)
#--------------------------------
HH = True
#--------------------------------
WW = False

WW_ll = False
WW_tt = False
#********************************
# Filters (choose one):
#--------------------------------
fatJetFiltered = False
#--------------------------------
viljaFiltered=True

# These are modifications of the original viljaFilter.
# LaurilaMod has looser filters
# SalomaaMod has more variables but same filters (I recommend using this)

viljaFiltered_original = False
viljaFiltered_LaurilaMod = False
viljaFiltered_SalomaaMod = True
#********************************
# How much data (you can ignore or delete these if you want):
#--------------------------------
HH_one = False
viljaFilteredFromAll = False
#--------------------------------
bigData = False
allData = False

# Create dataframe from NanoAOD files:

In [7]:
# Some information of the datafiles:
#********************************
# HH file path:
# /eos/cms/store/cmst3/group/vhcc/nanoTuples/v2_30Apr2020/2018/mc/VBF_HH_CV_1_C2V_0_C3_1_dipoleRecoilOff-TuneCP5_PSweights_13TeV-madgraph-pythia8/NanoTuples-30Apr2020_RunIIAutumn18MiniAOD-102X_v15-v2/200601_133715/0000/

# FILE CODE: VBF HH CV 1 C2V 0 C3 1 dipoleRecoilOff-TuneCP5 PSweights 13TeV-madgraph-pythia8 0.009169
#--------------------------------
# WW_ll file path:
# /eos/cms/store/cmst3/group/vhcc/nanoTuples/v2_30Apr2020/2016/mc/WWjj_SS_ll_hadronicMG272_WPt200Inf/NanoTuples-30Apr2020_kirschen-FullSim_94X-2016-MINIAODSIM-bd3e7bcff6c9bcad356ea4ed7e4f08b4/210708_131326/0000/
#--------------------------------
# WW_tt file path:
# /eos/cms/store/cmst3/group/vhcc/nanoTuples/v2_30Apr2020/2016/mc/WWjj_SS_tt_hadronicMG272_WPt200Inf/NanoTuples-30Apr2020_kirschen-FullSim_94X-2016-MINIAODSIM-bd3e7bcff6c9bcad356ea4ed7e4f08b4/210708_131251/0000/
#********************************
# If "the file is not found" use this in front of the path:
# root://xrootd-cms.infn.it/
#********************************

# Creating the RDataframe using chaining of the files
if HH:
    
    chain = ROOT.TChain('Events')

    path = '/eos/cms/store/cmst3/group/vhcc/nanoTuples/v2_30Apr2020/2018/mc/'
    path += 'VBF_HH_CV_1_C2V_0_C3_1_dipoleRecoilOff-TuneCP5_PSweights_13TeV-madgraph-pythia8/'
    path += 'NanoTuples-30Apr2020_RunIIAutumn18MiniAOD-102X_v15-v2/200601_133715/0000/'
    
    # These determine how much data you want. You can tweak these easily here if you want.
    if HH_one:
        r = range(1,2,1)
    elif viljaFilteredFromAll:
        r = range(1, 120, 1)
    elif viljaFiltered:
        r = range(1, 13, 1)
    elif fatJetFiltered:
        r = range(1, 17, 1)
    else:
        r = range(1,2,1)
    for i in r:
        path_i = path + 'nano_' + str(i) + '.root'
        chain.AddFile(path_i)
    
elif WW:
    
    chain = ROOT.TChain('Events')
    
    path = '/eos/cms/store/cmst3/group/vhcc/nanoTuples/v2_30Apr2020/2016/mc/'
    
    # Here also you can choose the amount of the data.
    if WW_ll:
        path += 'WWjj_SS_ll_hadronicMG272_WPt200Inf/'
        path += 'NanoTuples-30Apr2020_kirschen-FullSim_94X-2016-MINIAODSIM-bd3e7bcff6c9bcad356ea4ed7e4f08b4/210708_131326/0000/'
        for i in range(1, 501, 1):
            path_i = path + 'nano_' + str(i) + '.root'
            chain.AddFile(path_i)
    elif WW_tt:
        path += 'WWjj_SS_tt_hadronicMG272_WPt200Inf/'
        path += 'NanoTuples-30Apr2020_kirschen-FullSim_94X-2016-MINIAODSIM-bd3e7bcff6c9bcad356ea4ed7e4f08b4/210708_131251/0000/'
        for i in range(1, 206, 1):
            path_i = path + 'nano_' + str(i) + '.root'
            chain.AddFile(path_i)
            if i == 1:
                print(path_i)

df = ROOT.RDataFrame(chain)

## FindClosest (c++ code):

In [8]:
# Run always
# Define c++ functions that find the closest target using eta and phi coordinates

%run findClosestFunctions.py

## bGenPart (c++ code):

In [9]:
# Run always
# Define c++ functions related to finding last GenParts and best GenParts (bGenPart)

%run bGenPartFunctions.py

# -------------------------------------

# Time to filter the data df --> df0 (1/2):

In [10]:
#*************************************
# fatJetFiltered:

# Filter dataframe to contain >= two (>400 pt fat jets) with one over 500 pt
# Select only events with at least two AK8 jets
#-------------------------------------
# viljaFiltered:

# First use the viljaFilter
# After that filters:
# 1)    Events with >= 2 nGenJets
# 2)    Events with strong matched VBF jets (only "strong events")
# 3)    Events with different bJets (a.k.a. separate)
#*************************************

if fatJetFiltered:
    df_cut_nak8 = df.Filter('nFatJet >= 2', 'Events with >=2 AK8 jets')
    df_cut_ak8pt = df_cut_nak8.Filter('Sum(FatJet_pt > 400.0) >= 2', 'Events with >=2 AK8 jets with pT > 400 GeV')
    df_cut_leadingak8pt = df_cut_ak8pt.Filter('Sum(FatJet_pt > 500.0) > 0', 'Events with >=1 AK8 jets with pT > 500 GeV')
    df_cut_leadingak8pt.Report().Print()
    df0 = df_cut_leadingak8pt
elif viljaFiltered:
    #-------------------------------------
    if viljaFiltered_original:
        %run viljaFilter.py
    elif viljaFiltered_LaurilaMod:
        %run viljaFilter_LaurilaMod.py
    elif viljaFiltered_SalomaaMod: 
        %run viljaFilter_SalomaaMod.py
    #-------------------------------------
    if HH_one:
        df_viljaFiltered = filtersAndcuts(df, 36000, 0.009169)
    elif viljaFiltered and (not viljaFilteredFromAll):
        df_viljaFiltered = filtersAndcuts(df, 292000, 0.009169)
    #-------------------------------------
    # filtering will continue soon...
    #-------------------------------------
else:
    df0 = df

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available
Warning in <TClass::Init>: no dictionary for class __pair_base<edm::Hash<1>,edm::ParameterSetBlob> is available


#### Define bGenPart, bGenJet and bJet variables:

In [11]:
# Run always
# Defining variables using the functions in findClosestFunctions.py and bGenPartFunctions.py

%run def_bVariables.py

if not viljaFiltered:
    df0 = defbVariables(df0)
else:
    df_viljaFiltered_with_bVariables = defbVariables(df_viljaFiltered)

# Filtering using nB_strong (2/2):

In [12]:
#-------------------------------------
# ... filtering continues:

if viljaFiltered:
    df_viljaFiltered_with_GenJets_Jets = df_viljaFiltered_with_bVariables.Filter('nGenJet >= 2', 'Events with >= 2 nGenJets')
    df_viljaFiltered_strong = df_viljaFiltered_with_GenJets_Jets.Filter('nB_strong == 2', 'Events with strong matched VBF jets')
    df_viljaFiltered_strong_sep = df_viljaFiltered_strong.Filter('bJet_eta[0] != bJet_eta[1]', 'Events with different bJets')
    df_viljaFiltered_strong_sep.Report().Print()
    df0 = df_viljaFiltered_strong_sep
#-------------------------------------

Events with >=2 AK8 jets: pass=119845     all=292000     -- eff=41.04 % cumulative eff=41.04 %
Preselection of subleading jet pt: pass=75527      all=119845     -- eff=63.02 % cumulative eff=25.87 %
Preselection of leading jet pt: pass=62550      all=75527      -- eff=82.82 % cumulative eff=21.42 %
Events passing at least one trigger: pass=53930      all=62550      -- eff=86.22 % cumulative eff=18.47 %
Events with >=2 good AK8 jets with pT > 400 GeV: pass=40416      all=53930      -- eff=74.94 % cumulative eff=13.84 %
Events with >0 good AK8 jets with pT > 500 GeV: pass=34803      all=40416      -- eff=86.11 % cumulative eff=11.92 %
Events with >=2 good AK8 jets with deltaEta < 2.0: pass=32775      all=34803      -- eff=94.17 % cumulative eff=11.22 %
Events with >=2 good AK8 jets with deltaPhi > 2.6: pass=32197      all=32775      -- eff=98.24 % cumulative eff=11.03 %
Events with >=2 good AK8 jets with 145 > ms > 100 GeV: pass=19698      all=32197      -- eff=61.18 % cumulative eff=6.7

# -------------------------------------

## Define PrintStats functions:

In [13]:
# Run always
# This is to investigate the success rates of the matchings

%run printStatsFunctions.py

# Examples use of use:

# printStats(df0, ['nB', 'nB_weak', 'nB_strong'])

# OR

#printStats(df0, ['nB_etaDetectable', 'nB_etaDetectable_weak', 'nB_etaDetectable_strong'])
#printStats(df0, ['nB_GenJetDetectable', 'nB_GenJetDetectable_weak', 'nB_GenJetDetectable_strong'])
#printStats(df0, ['nB_JetDetectable', 'nB_JetDetectable_weak', 'nB_JetDetectable_strong'])

# -------------------------------------

## Define PrintEvents functions and global variables:

##### Load and run these next cells if you want to study specific events, or events one by one (very nice stuff to investigate)

In [14]:
# %load printEventsFunctions.py

In [15]:
#df0 = df0.Define('GenPart_statusFlags_intRVec', 'intRVec_to_intRVecRVec(GenPart_statusFlags)')

In [16]:
# %load printEventsGlobVariables.py

In [17]:
# Examples of use:

# printEvents(50,
#             showVBFCandidates=False,
#             onlyNotWeaks=True, onlyNotWeaks_and_bigPt=False,
#             printShort=True, printIdx=False, printIdxStatus=False, printCoordinateAndPt=True)

# OR (after the shortNotEqualIndexList definition below)

# printEvents(shortNotEqualIndexList[:10],
#             showVBFCandidates=True,
#             onlyNotWeaks=False, onlyNotWeaks_and_bigPt=False,
#             printShort=True, printIdx=False, printIdxStatus=False, printCoordinateAndPt=True)

# -------------------------------------

##### Q1 vs Q2 comparison variables:

$\Delta\eta_{01}=\eta[0]-\eta[1]$ <br>
$|\Delta\eta_{01}|$<br>
$|\Delta|\eta_{01}||$<br>
----------------------------------- <br>
$\Delta\phi_{01}=\phi[0]-\phi[1]$ <br>
$|\Delta\phi_{01}|$ <br>
----------------------------------- <br>
$\Delta {p_T}_{01}=p_T[0]-p_T[1]$ <br>
$|\Delta p_{T,01}|$ <br>
$|\Delta p_{T,01}/p_T[1]|$ <br>
----------------------------------- <br>
$p_{T,0,rel}=\frac{p_T[0]}{p_T[0]+p_T[1]}$ <br>
$p_{T,1,rel}=\frac{p_T[1]}{p_T[0]+p_T[1]}$ <br>
$min(p_{T,0,rel}, p_{T,1,rel})$

In [18]:
# Run this if you have unfiltered or fatjet-filtered events and you want to proceed with "strong events" (two strong matched jets) only

#%run def_Q1_Q2_variables.py
#df0_strong = df0.Filter('nB_strong == 2', 'nB_strong == 2')
#df0_strong = define_Q1_Q2_variables(df_strong)

# -------------------------------------

# Run and time your code after this:

# -------------------------------------

In [19]:
# For viljaFiltered dataset to check how often VBFCandidate equals bJet (which is the goal)
# This also returns notEqualIndexList which are the event indices that VBFC differs from bJet
# These statistics are at the end of the pdf file
# You might want to run this to get indexList for the printEvents() functions (to check why it's not the same)

if viljaFiltered:

    npdf = df0.AsNumpy(['bJet_eta', 'VBFCandidates_eta'])
    notEqualIndexList = []
    
    def inVBFList(q, i):
        VBFjet = npdf['VBFCandidates_eta'][i][q]
        bJetList = [npdf['bJet_eta'][i][0], npdf['bJet_eta'][i][1]]
        inList = VBFjet in bJetList
        return inList

    m = len(npdf['bJet_eta'])

    s0 = 0
    s1 = 0
    s2 = 0

    for i in range(m):
        c0 = inVBFList(0, i)
        c1 = inVBFList(1, i)
        c2 = c0 and c1

        if c0:
            s0 += 1
        if c1:
            s1 += 1

        if c2:
            s2 += 1
        else:
            notEqualIndexList.append(i)
    print('(#VBFC[0] correct, #VBFC[1] correct, #VBFC correct, #both correct)')
    print((s0, s1, (s0 + s1), s2))
    print((round(s0/m, 3), round(s1/m, 3), round((s0 + s1)/(2*m), 3), round(s2/m, 3)))

(#VBFC[0] correct, #VBFC[1] correct, #VBFC correct, #both correct)
(3276, 3008, 6284, 2940)
(0.978, 0.898, 0.938, 0.878)


In [20]:
# Short version of the notEqualIndexList created above

shortNotEqualIndexList = [0, 8, 31, 40, 42, 47, 49, 58, 62, 72, 73, 88, 93, 114, 123, 136, 137, 143, 151, 157, 180, 203, 213, 224, 229, 240, 252, 257]

# ---------------------------------

# Machine learning stuff (%load and run):

In [21]:
#%load machineLearning_dataframe.py

In [22]:
#%load machineLearning_stats.py

In [23]:
#%load machineLearning_plots.py

# ---------------------------------

In [24]:
# If wanted these "time stamps" can be thrown in the code

In [25]:
t0 = time.time()
print(str(round(t0-t_init)) + ' s = ' + str(round((t0-t_init)/60)) + ' min')

73 s = 1 min


In [26]:
t1 = time.time()
print(str(round(t1-t0)) + ' s = ' + str(round((t1-t0)/60)) + ' min')
print(str(round(t1-t_init)) + ' s = ' + str(round((t1-t_init)/60)) + ' min')

0 s = 0 min
73 s = 1 min


In [27]:
t2 = time.time()
print(str(round(t2-t1)) + ' s = ' + str(round((t2-t1)/60)) + ' min ')
print(str(round(t2-t_init)) + ' s = ' + str(round((t2-t_init)/60)) + ' min ')

0 s = 0 min 
73 s = 1 min 


In [28]:
t3 = time.time()
print(str(round(t3-t2)) + ' s = ' + str(round((t3-t2)/60)) + ' min ')
print(str(round(t3-t_init)) + ' s = ' + str(round((t3-t_init)/60)) + ' min ')

0 s = 0 min 
74 s = 1 min 


In [29]:
t4 = time.time()
print(str(round(t4-t3)) + ' s = ' + str(round((t4-t3)/60)) + ' min ')
print(str(round(t4-t_init)) + ' s = ' + str(round((t4-t_init)/60)) + ' min ')

0 s = 0 min 
74 s = 1 min 


In [30]:
t5 = time.time()
print(str(round(t5-t4)) + ' s = ' + str(round((t5-t4)/60)) + ' min')
print(str(round(t5-t_init)) + ' s = ' + str(round((t5-t_init)/60)) + ' min')

0 s = 0 min
74 s = 1 min


In [31]:
t6 = time.time()
print(str(round(t6-t5)) + ' s = ' + str(round((t6-t5)/60)) + ' min ')
print(str(round(t6-t_init)) + ' s = ' + str(round((t6-t_init)/60)) + ' min ')

0 s = 0 min 
74 s = 1 min 


# ---------------------------------

In [32]:
t_final = time.time()
print(str(round(t_final-t6)) + ' s = ' + str(round((t_final-t6)/60)) + ' min ')
print(str(round(t_final-t_init)) + ' s = ' + str(round((t_final-t_init)/60)) + ' min ')

0 s = 0 min 
74 s = 1 min 
